In [ ]:
#!pip install ipywidgets

from ipywidgets import interact, widgets, GridspecLayout

def generate_playfair_matrix(key):
    key = key.upper().replace("J", "I")  # Convert to uppercase and replace 'J' with 'I'
    key = "".join(sorted(set(key), key=key.index))  # Remove duplicates and preserve order
    alphabet = "ABCDEFGHIKLMNOPQRSTUVWXYZ"  # Playfair excludes 'J'
    key += alphabet

    playfair_matrix = []
    for i in range(5):
        row = key[i * 5:(i + 1) * 5]
        playfair_matrix.append(list(row))

    return playfair_matrix

def playfair_encrypt(plaintext, key):
    playfair_matrix = generate_playfair_matrix(key)
    ciphertext = ""

    pairs = []
    i = 0
    while i < len(plaintext):
        pair = plaintext[i:i + 2]
        if len(pair) == 1:
            pair += 'X'
            i -= 1
        pairs.append(pair)
        i += 2

    for pair in pairs:
        char1, char2 = pair[0], pair[1]
        row1, col1 = find_position(playfair_matrix, char1)
        row2, col2 = find_position(playfair_matrix, char2)

        if row1 == row2:  # Same row
            ciphertext += playfair_matrix[row1][(col1 + 1) % 5] + playfair_matrix[row2][(col2 + 1) % 5]
        elif col1 == col2:  # Same column
            ciphertext += playfair_matrix[(row1 + 1) % 5][col1] + playfair_matrix[(row2 + 1) % 5][col2]
        else:  # Form a rectangle
            ciphertext += playfair_matrix[row1][col2] + playfair_matrix[row2][col1]

    return ciphertext

def playfair_decrypt(ciphertext, key):
    playfair_matrix = generate_playfair_matrix(key)
    plaintext = ""

    pairs = [ciphertext[i:i + 2] for i in range(0, len(ciphertext), 2)]

    for pair in pairs:
        char1, char2 = pair[0], pair[1]
        row1, col1 = find_position(playfair_matrix, char1)
        row2, col2 = find_position(playfair_matrix, char2)

        if row1 == row2:  # Same row
            plaintext += playfair_matrix[row1][(col1 - 1) % 5] + playfair_matrix[row2][(col2 - 1) % 5]
        elif col1 == col2:  # Same column
            plaintext += playfair_matrix[(row1 - 1) % 5][col1] + playfair_matrix[(row2 - 1) % 5][col2]
        else:  # Form a rectangle
            plaintext += playfair_matrix[row1][col2] + playfair_matrix[row2][col1]

    return plaintext

def find_position(matrix, char):
    if char == 'J':
        char = 'I'

    for i in range(5):
        for j in range(5):
            if matrix[i][j] == char:
                return i, j
    # Handle case when character is not found
    return -1, -1

def display_matrix(matrix):
    layout = GridspecLayout(5, 5, height='auto')
    for i in range(5):
        for j in range(5):
            layout[i, j] = widgets.Label(value=matrix[i][j], layout=widgets.Layout(width='40px', height='40px'))

    return layout

def on_button_click(b):
    message = input_text.value.upper()
    key = key_input.value.upper()

    if encrypt_radio.value:
        result_text.value = playfair_encrypt(message, key)
    else:
        result_text.value = playfair_decrypt(message, key)

    matrix_output.clear_output(wait=True)
    with matrix_output:
        display(display_matrix(generate_playfair_matrix(key)))

# Create widgets
input_text = widgets.Textarea(value='HELLO', description='Message:')
key_input = widgets.Text(value='KEY', description='Key:')
encrypt_radio = widgets.RadioButtons(options=['Encrypt', 'Decrypt'], description='Operation:')
encrypt_button = widgets.Button(description='Encrypt/Decrypt')
result_text = widgets.Textarea(value='', description='Result:', disabled=True)
matrix_output = widgets.Output()

# Set up interactivity
encrypt_button.on_click(on_button_click)

# Display widgets
widgets.VBox([input_text, key_input, encrypt_radio, encrypt_button, result_text, matrix_output])
